# 07: CI Integration — Architecture Enforcement in Pipelines

KindScript is designed for CI/CD. Every architectural violation returns exit code `1` — just like `tsc` returns `1` for type errors.

This notebook demonstrates:
1. Exit code behavior for clean vs. violation projects
2. Diagnostic output format (compatible with TypeScript tooling)
3. GitHub Actions / CI configuration examples
4. Pre-commit hook setup
5. Migrating existing projects

## Setup

In [ ]:
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";

async function ksc(...args: string[]): Promise<{ code: number; stdout: string; stderr: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  return {
    code,
    stdout: new TextDecoder().decode(stdout).trim(),
    stderr: new TextDecoder().decode(stderr).trim(),
  };
}

const BOILERPLATE = `
interface Kind<N extends string = string> {
  readonly kind: N;
  readonly location: string;
}
interface ContractConfig {
  noDependency?: [string, string][];
  purity?: string[];
}
function defineContracts<_T = unknown>(config: ContractConfig): ContractConfig {
  return config;
}
`.trimStart();

console.log("Setup complete.");

---

## 1. Exit Code Behavior

| Scenario | Exit code |
|----------|----------|
| All contracts satisfied | `0` |
| One or more violations | `1` |
| Missing config file | `1` |
| Invalid definitions | `1` |

This is the same convention as `tsc`, `eslint`, and `jest`.

In [ ]:
// Create two projects: one clean, one with a violation
function makeCleanProject(): string {
  const dir = Deno.makeTempDirSync({ prefix: "ksc-ci-clean-" });
  Deno.mkdirSync(`${dir}/src/domain`, { recursive: true });
  Deno.mkdirSync(`${dir}/src/infrastructure`, { recursive: true });
  Deno.writeTextFileSync(`${dir}/src/domain/entity.ts`, `export interface User { id: string; }\n`);
  Deno.writeTextFileSync(`${dir}/src/infrastructure/repo.ts`, `import { User } from '../domain/entity';\nexport function save(u: User) {}\n`);
  Deno.writeTextFileSync(`${dir}/tsconfig.json`, JSON.stringify({
    compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
    include: ["src/**/*.ts"],
  }, null, 2));
  Deno.writeTextFileSync(`${dir}/kindscript.json`, JSON.stringify({ definitions: ["architecture.ts"] }, null, 2));
  Deno.writeTextFileSync(`${dir}/architecture.ts`, BOILERPLATE + `
export interface App extends Kind<"App"> {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}
export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export const app: App = {
  kind: "App",
  location: "src",
  domain: { kind: "DomainLayer", location: "src/domain" },
  infrastructure: { kind: "InfrastructureLayer", location: "src/infrastructure" },
};

export const contracts = defineContracts<App>({
  noDependency: [["domain", "infrastructure"]],
});
`);
  return dir;
}

function makeViolationProject(): string {
  const dir = makeCleanProject();
  // Add a forbidden import in domain
  Deno.writeTextFileSync(`${dir}/src/domain/entity.ts`, `
import { save } from '../infrastructure/repo';
export interface User { id: string; }
export function createAndSave() { save({ id: '1' }); }
`.trimStart());
  return dir;
}

console.log("Project factories ready.");

In [ ]:
// Exit code 0: clean project
const clean = makeCleanProject();
const cleanResult = await ksc("check", clean);
console.log("STDOUT:", cleanResult.stdout);
console.log("STDERR:", cleanResult.stderr);
console.log("EXIT CODE:", cleanResult.code);
console.log();

// Exit code 1: violation
const violation = makeViolationProject();
const violationResult = await ksc("check", violation);
console.log("STDOUT:", violationResult.stdout);
console.log("STDERR:", violationResult.stderr);
console.log("EXIT CODE:", violationResult.code);
console.log();

// Exit code 1: missing config
const noConfig = Deno.makeTempDirSync({ prefix: "ksc-ci-noconfig-" });
const noConfigResult = await ksc("check", noConfig);
console.log("STDOUT:", noConfigResult.stdout);
console.log("STDERR:", noConfigResult.stderr);
console.log("EXIT CODE:", noConfigResult.code);

Deno.removeSync(clean, { recursive: true });
Deno.removeSync(violation, { recursive: true });
Deno.removeSync(noConfig, { recursive: true });

---

## 2. Diagnostic Output Format

KindScript diagnostics follow the TypeScript diagnostic format:

```
file:line:column - error KSXXXXX: message
  Contract 'name' (type) defined at location
```

This means existing tools that parse TypeScript errors (VS Code problem matchers, CI log parsers) work out of the box.

In [ ]:
// Show the exact diagnostic format
const demo = makeViolationProject();
const result = await ksc("check", demo);

console.log("\n=== Diagnostic format breakdown ===");
const lines = (result.stdout + result.stderr).split("\n");
for (const line of lines) {
  if (line.includes("error KS")) {
    console.log("Error line:", line);
    // Parse the format
    const match = line.match(/(.+):(\d+):(\d+) - error (KS\d+): (.+)/);
    if (match) {
      console.log("  File:    ", match[1]);
      console.log("  Line:    ", match[2]);
      console.log("  Column:  ", match[3]);
      console.log("  Code:    ", match[4]);
      console.log("  Message: ", match[5]);
    }
  }
  if (line.includes("Contract '")) {
    console.log("Context:  ", line.trim());
  }
  if (line.includes("violation")) {
    console.log("Summary:  ", line.trim());
  }
}

Deno.removeSync(demo, { recursive: true });

---

## 3. GitHub Actions Example

Add KindScript to your CI pipeline alongside `tsc` and your test runner:

```yaml
# .github/workflows/ci.yml
name: CI
on: [push, pull_request]

jobs:
  check:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
        with:
          node-version: 20
      - run: npm ci

      # Type check
      - run: npx tsc --noEmit

      # Architecture check
      - run: npx ksc check

      # Tests
      - run: npm test
```

If `ksc check` finds a violation, the job fails with exit code 1 — just like a failing `tsc` or test.

## 4. Pre-commit Hook

Catch violations before they reach CI:

```bash
# .git/hooks/pre-commit (make executable: chmod +x)
#!/bin/sh
npx ksc check || exit 1
```

Or with [Husky](https://typicode.github.io/husky/):

```bash
# .husky/pre-commit
npx ksc check
```

Or with [lint-staged](https://github.com/lint-staged/lint-staged) (only checks when `.ts` files change):

```json
{
  "lint-staged": {
    "*.ts": "ksc check"
  }
}
```

## 5. Utility Commands for Scripts

Two commands useful for CI scripting:

In [ ]:
// Version check — useful for pinning in CI
const version = await ksc("--version");
console.log("Version:", version.stdout || version.stderr);

In [ ]:
// Help — shows all available commands
const help = await ksc("--help");
console.log(help.stdout || help.stderr);

---

## 6. Migration Workflow

For projects adopting KindScript for the first time:

```bash
# Step 1: Detect what you have
npx ksc init --detect

# Step 2: Generate definitions with inferred contracts
npx ksc infer --write

# Step 3: Check — may find existing violations
npx ksc check

# Step 4: Fix violations or relax contracts
# Edit architecture.ts to remove contracts you're not ready to enforce

# Step 5: Add to CI
# Add `npx ksc check` to your pipeline
```

**Gradual adoption:** Start with just `noDependency` for your most critical boundary (e.g., domain must not import from infrastructure). Add more contracts as you clean up the codebase.

In [ ]:
// Demonstrate the migration workflow
const migrate = Deno.makeTempDirSync({ prefix: "ksc-migrate-" });
Deno.mkdirSync(`${migrate}/src/domain`, { recursive: true });
Deno.mkdirSync(`${migrate}/src/application`, { recursive: true });
Deno.mkdirSync(`${migrate}/src/infrastructure`, { recursive: true });

Deno.writeTextFileSync(`${migrate}/src/domain/entity.ts`, `export interface User { id: string; }\n`);
Deno.writeTextFileSync(`${migrate}/src/application/handler.ts`, `import { User } from '../domain/entity';\nexport function handle(u: User) {}\n`);
Deno.writeTextFileSync(`${migrate}/src/infrastructure/repo.ts`, `import { User } from '../domain/entity';\nexport function save(u: User) {}\n`);
Deno.writeTextFileSync(`${migrate}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
  include: ["src/**/*.ts"],
}, null, 2));

console.log("=== Step 1: Detect ===");
const detectResult = await ksc("init", "--detect", migrate);
console.log(detectResult.stderr || detectResult.stdout || "(no output)");
console.log("Exit code:", detectResult.code);

console.log("\n=== Step 2: Infer + write ===");
const inferResult = await ksc("infer", "--write", migrate);
console.log(inferResult.stderr || inferResult.stdout || "(no output)");
console.log("Exit code:", inferResult.code);

console.log("\n=== Step 3: Check ===");
const checkResult = await ksc("check", migrate);
console.log(checkResult.stderr || checkResult.stdout || "(no output)");
console.log("Exit code:", checkResult.code);

console.log("\nMigration complete. Architecture is now enforced.");

Deno.removeSync(migrate, { recursive: true });

---

## Summary

| Integration point | How |
|---|---|
| **CI pipeline** | `npx ksc check` — fails with exit code 1 on violations |
| **Pre-commit hook** | `npx ksc check \|\| exit 1` |
| **VS Code** | TypeScript language service plugin (see architecture docs) |
| **npm scripts** | `"arch-check": "ksc check"` in package.json |

KindScript follows the Unix convention: exit 0 = success, exit 1 = failure. It outputs diagnostics in TypeScript's format. It works with any CI system that can run `node`.